# Speed Test

This notebook details the methodology used to compare the hardware random number generation to the software random number generation.

In [ ]:
from pynq import Overlay
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import random as r

ol = Overlay("base_new.bit")            # TCL file is parsed

x = 512                             # Define number of columns in image
y = 384                             # Define number of rows in image
numVals = x*y                       # Calculate required number of values

## Hardware Generation

In this section, the required number of values are generated by our custom hardware IP.

In [ ]:
seed = 0x00                           # Define each of the memory addresses of the IP block
reset = 0x04
load = 0x08
enable = 0x0C
rand = 0x10
values = []                           # Create an empty array to store the generated random numbers

LFSR = ol.uniform_0                   # Make the IP block easier to address

LFSR.write(reset, 1)                  # Reset LFSR block

LFSR.write(reset, 0)                  # Unassert reset pin
LFSR.write(enable, 1)                 # Allow LFSR to operate
LFSR.write(load, 1)                   # Load seed int LFSR block
LFSR.write(seed, 5635)                # Write seed value to Din pin of LFSR

LFSR.write(load, 0)                   # Allow LFSR to update current value

start = time.time()                   # Store time before generating random numbers

for i in range(numVals):              # Iterate for required number of values
    values.append(LFSR.read(rand))    # append values to values array
    
process = time.time() - start         # Calculate time taken to generate required numbers
print(process)                        # Output time taken to generate required numbers

## Hardware Histogram Plot

In this section, a histogram is plotted to graphically display the distribution of generated values.

In [ ]:
plt.hist(values)                       # Plot histogram of generated numbers
plt.xlabel('Value')
plt.ylabel('Occurrence')
plt.title('Histogram')

## Hardware Distribution Image

In this section, The generated values are mapped to a value between 0 and 255 to demonstrate the distribution as a greyscale image.

In [ ]:
rgb = []                                                            # Create empty array to store greyscale tuples
greyArr = []                                                        # Create empty array to store greyscale values
top = min(values)                                                   # Find minimum generated value
bot = max(values)                                                   # Find maximum generated value

for i in range(len(values)):                                        # Iterate through all generated values
    grey = int((values[i] - bot) * (255 - 0) / (top - bot) + 0)     # Quantize values between 0 and 255
    greyArr.append(grey)                                            # Append greyscale value to array
    rgb.extend([grey,grey,grey])                                    # Fill greyscale tuple array

image = Image.frombytes('RGB', (x,y), bytes(rgb))                   # Load greyscale data onto new image
image.save("Test.jpg",'JPEG')                                       # Save created image
image                                                               # Display image in notebook

## Software Generation

In this section, software generation methods are used to compare against the hardware method detailed previously. The 'random' library is used as it is comes with the Pynq image and it comparable in complexity to the hardware method used.

In [ ]:
values = []                            # Create empty array to store generated values in
r.seed(5635)                           # Use same seed as hardware for fairness

start = time.time()                    # Store current time before generation of random numbers

for i in range(numVals):               # Iterate for required number of values
    values.append(r.randint(0,255))    # Append generated value to array
    
process = time.time() - start          # Calculate time taken to generate required number of values
print(process)                         # Print time taken in notebook

## Software Histogram Plot

In this section, a histogram is plotted of the software generated values to provide a direct comparison with the hardware generated values.

In [ ]:
plt.hist(values)                       # Plot histogram of generated numbers
plt.xlabel('Value')
plt.ylabel('Occurrence')
plt.title('Histogram')

## Software Distribution Image

In this section, The software generated values are used to generate a greyscale distribution image in the same way as carried out on the hardware previously.

In [ ]:
rgb = []                                             # Create an empty array to store the greyscale tuples

for i in range(len(values)):                         # Iterate through all generated values
    rgb.extend([values[i],values[i],values[i]])      # Add the greyscale tuples to the array

image = Image.frombytes('RGB', (x,y), bytes(rgb))    # Add the greyscale tuples to a new image
image.save("Test.jpg", 'JPEG')                       # Save the created image
image                                                # Display the created image in the notebook